<a href="https://colab.research.google.com/github/shunK927/2023-10-10/blob/main/site/en/gemini-api/docs/vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [50]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Explore vision capabilities with the Gemini API

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs/vision"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

The Gemini API can run inference on images and videos passed to it. When passed an image, a series of images, or a video, Gemini can:

*   Describe or answer questions about the content
*   Summarize the content
*   Extrapolate from the content

This tutorial demonstrates some possible ways to prompt the Gemini API with
images and video input. All output is text-only.

## Setup

Before you use the File API, you need to install the Gemini API SDK package and configure an API key. This section describes how to complete these setup steps.

### Install the Python SDK and import packages

The Python SDK for the Gemini API is contained in the [google-generativeai](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip.

In [51]:
!pip install -q -U google-generativeai

Import the necessary packages.

In [52]:
import google.generativeai as genai
from IPython.display import Markdown

### Set up your API key

The File API uses API keys for authentication and access. Uploaded files are associated with the project linked to the API key. Unlike other Gemini APIs that use API keys, your API key also grants access to data you've uploaded to the File API, so take extra care in keeping your API key secure. For more on keeping your keys
secure, see [Best practices for using API
keys](https://support.google.com/googleapi/answer/6310037).

Store your API key in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or are unfamiliar with Colab Secrets, refer to the [Authentication quickstart](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb).

In [53]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Prompting with images

In this tutorial, you will upload images using the File API or as inline data and generate content based on those images.

### Technical details (images)
Gemini 1.5 Pro and Flash support a maximum of 3,600 image files.

Images must be in one of the following image data [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):

-   PNG - `image/png`
-   JPEG - `image/jpeg`
-   WEBP - `image/webp`
-   HEIC - `image/heic`
-   HEIF - `image/heif`

Each image is equivalent to 258 tokens.

While there are no specific limits to the number of pixels in an image besides the model’s context window, larger images are scaled down to a maximum resolution of 3072x3072 while preserving their original aspect ratio, while smaller images are scaled up to 768x768 pixels. There is no cost reduction for images at lower sizes, other than bandwidth, or performance improvement for images at higher resolution.

For best results:

*   Rotate images to the correct orientation before uploading.
*   Avoid blurry images.
*   If using a single image, place the text prompt after the image.

### Upload an image file using the File API

Use the File API to upload an image of any size. (Always use the File API when the combination of files and system instructions that you intend to send is larger than 20 MB.)

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but cannot be downloaded from the API. It is available at no cost in all regions where the Gemini API is available.

Start by downloading this [sketch of a jetpack](https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg).

In [54]:
!curl -o jetpack.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  2001k      0 --:--:-- --:--:-- --:--:-- 2006k


Upload the image using [`media.upload`](https://ai.google.dev/api/rest/v1beta/media/upload) and print the URI, which is used as a reference in Gemini API calls.

In [55]:
# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="jetpack.jpg",
                            display_name="Jetpack drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

Uploaded file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/kzilazo5pvo1


The `response` shows that the File API stored the specified `display_name` for the uploaded file and a `uri` to reference the file in Gemini API calls. Use `response` to track how uploaded files are mapped to URIs.

Depending on your use case, you can also store the URIs in structures such as a `dict` or a database.

### Verify image file upload and get metadata

You can verify the API successfully stored the uploaded file and get its metadata by calling [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) through the SDK. Only the `name` (and by extension, the `uri`) are unique. Use `display_name` to identify files only if you manage uniqueness yourself.

In [56]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

Retrieved file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/kzilazo5pvo1


Depending on your use case, you can store the URIs in structures, such as a `dict` or a database.

### Prompt with the uploaded image and text

After uploading the file, you can make GenerateContent requests that reference the File API URI. Select the generative model and provide it with a text prompt and the uploaded image.

In [58]:
# Choose a Gemini API model.
model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest")

print("Uploaded file URI:", sample_file.uri)

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Describe how this product might be manufactured."])

Markdown(">" + response.text)

Uploaded file URI: https://generativelanguage.googleapis.com/v1beta/files/kzilazo5pvo1


>Here's a breakdown of how the Jetpack Backpack might be manufactured, based on the provided sketch:

**1. Backpack Body:**

* **Material Selection:**  The backpack's main body would likely be made from a lightweight yet durable material, possibly nylon or ripstop fabric for strength and water resistance.  The choice would depend on balancing weight and robustness.  Consideration might be given to using recycled or sustainable materials to align with the "green/clean" aspect.

* **Construction:**  The backpack body would be constructed using industrial sewing machines.  Panels of fabric would be cut using computer-aided design (CAD) and cutting machines for precision.  Seams would be reinforced for strength.  The design aims for a standard backpack aesthetic to avoid looking bulky or unusual.

* **Zippers and Fasteners:**  Standard zippers and potentially other fasteners would be added to allow access to the main compartment and any other pockets.

**2. Strap Support:**

* **Padding:**  The straps would require padded material, likely foam or a similar cushioning material, to ensure comfort during wear.  This padding could be glued or sewn onto the straps.

* **Attachment:**  The straps would be securely attached to the backpack using strong stitching and possibly additional reinforcement materials (like bar tacking).


**3. Internal Components:**

* **Laptop Compartment:**  This compartment would need to be designed with the 18-inch laptop size in mind and likely use a protective lining or padding.

* **USB-C Charging System:**  This would require integrating a battery pack (potentially a lithium-ion battery) and USB-C charging ports. The battery pack would need protective casing and proper circuitry.  The manufacturing would involve soldering, circuit board assembly, and integration into the backpack.

**4. Retractable Boosters:**

* **Mechanism:**  The retractable boosters represent the most significant engineering challenge.  This would require miniaturized steam turbines (or a similar green energy-based propulsion system) with a retractable mechanism.  This mechanism could involve precisely engineered gears, motors, and actuators for safe and controlled deployment.

* **Steam Generation:**  A miniature, efficient steam generator would be needed, possibly using a small, clean-burning fuel source like ethanol or biofuel.  This would require expertise in thermodynamics and miniature engine design.

* **Fuel Tank:**  A compact, safe fuel tank would be needed to contain the fuel.  This needs to meet safety regulations and be seamlessly integrated into the backpack.


**5. Assembly:**

* Once all components are manufactured, they would need to be carefully assembled. This would involve a combination of manual assembly (stitching, attaching straps) and potentially automated processes (for electronics integration).  Quality control checks would be crucial at every stage.


**Overall Challenges:**

The biggest challenge would lie in the miniaturization and integration of the steam-powered retractable boosters.  The engineering demands for creating safe, efficient, and compact steam turbines would be significant.  Safety testing and regulatory compliance would also be extremely important before such a product could be marketed.


### Upload one or more locally stored image files

Alternatively, you can upload your own files. You can download and use our drawings of [piranha-infested waters](https://storage.googleapis.com/generativeai-downloads/images/piranha.jpg) and a [firefighter with a cat](https://storage.googleapis.com/generativeai-downloads/images/firefighter.jpg). First, save these files to your local directory.

Then click **Files** on the left sidebar. For each file, click the **Upload** button, then navigate to that file's location and upload it:

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files, as previously shown. Smaller files can instead be called locally from the Gemini API:


In [64]:
from google.colab import files
import PIL.Image

# ファイルのアップロード
uploaded = files.upload()

# アップロードされたファイル名を確認
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

    # 画像を開いて表示
    try:
        image = PIL.Image.open(filename)
        image.show()
    except Exception as e:
        print(f"Error opening image {filename}: {e}")


MessageError: RangeError: Maximum call stack size exceeded.

In [61]:
import PIL.Image

sample_file_2 = PIL.Image.open('piranha.jpg')

FileNotFoundError: [Errno 2] No such file or directory: '/content/piranha.jpg'

Note that these inline data calls don't include many of the features available via the File API, such as getting file metadata, [listing](https://colab.research.google.com/drive/19xeyIMZJIk7Zn9KW5_50iZYv8OfjApL5?resourcekey=0-3JZ6U8oAFX7hqeV7gAXshw#scrollTo=VosrkvAyrx-v&line=3&uniqifier=1), or [deleting](https://colab.research.google.com/drive/19xeyIMZJIk7Zn9KW5_50iZYv8OfjApL5?resourcekey=0-3JZ6U8oAFX7hqeV7gAXshw#scrollTo=diCy9BgjLqeS&line=1&uniqifier=1) files.

### Prompt with multiple images

You can provide the Gemini API with any combination of images and text that fit within the model's context window. This example provides one short text prompt and the three images previously uploaded.

In [ ]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

prompt = "Write an advertising jingle showing how the product in the first image could solve the problems shown in the second two images."

response = model.generate_content([prompt, sample_file, sample_file_2, sample_file_3])

Markdown(">" + response.text)

### Get bounding boxes

You can ask the model for the coordinates of bounding boxes for objects in images. For object detection, the Gemini model has been trained to provide
these coordinates as relative widths or heights in range `[0,1]`, scaled by 1000 and converted to an integer. Effectively, the coordinates given are for a
1000x1000 version of the original image, and need to be converted back to the dimensions of the original image.

In [ ]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

prompt = "Return a bounding box for the piranha. \n [ymin, xmin, ymax, xmax]"
response = model.generate_content([sample_file_2, prompt])

print(response.text)

To convert these coordinates to the dimensions of the original image:

1.    Divide each output coordinate by 1000.
1.    Multiply the x-coordinates by the original image width.
1.    Multiply the y-coordinates by the original image height.

## Prompting with video

In this tutorial, you will upload a video using the File API and generate content based on those images.

## Technical details (video)

Gemini 1.5 Pro and Flash support up to approximately an hour of video data.

Video must be in one of the following video format [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
  -   `video/mp4`
  -   `video/mpeg`
  -   `video/mov`
  -   `video/avi`
  -   `video/x-flv`
  -   `video/mpg`
  -   `video/webm`
  -   `video/wmv`
  -   `video/3gpp`

The File API service currently extracts image frames from videos at 1 frame per second (FPS) and audio at 1Kbps, single channel, adding timestamps every second. These rates are subject to change in the future for improvements in inference.

**NOTE:** The finer details of fast action sequences may be lost at the 1 FPS frame sampling rate. Consider slowing down high-speed clips for improved inference quality.

Individual frames are 258 tokens, and audio is 32 tokens per second. With metadata, each second of video becomes ~300 tokens, which means a 1M context window can fit slightly less than an hour of video.

To ask questions about time-stamped locations, use the format `MM:SS`, where the first two digits represent minutes and the last two digits represent seconds.

For best results:

*   Use one video per prompt.
*   If using a single video, place the text prompt after the video.

### Upload a video file to the File API

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but they cannot be downloaded using any API. It is available at no cost in all regions where the Gemini API is available.

The File API accepts video file formats directly. This example uses the short NASA film ["Jupiter's Great Red Spot Shrinks and Grows"](https://www.youtube.com/watch?v=JDi4IdtvDVE0). Credit: Goddard Space Flight Center (GSFC)/David Ladd (2018).

> "Jupiter's Great Red Spot Shrinks and Grows" is in the public domain and does not show identifiable people. ([NASA image and media usage guidelines.](https://www.nasa.gov/nasa-brand-center/images-and-media/))

Start by retrieving the short video:

In [ ]:
!wget https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4

Upload the video to the File API and print the URI.

In [ ]:
video_file_name = "GreatRedSpot.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

### Verify file upload and check state

Verify the API has successfully received the files by calling the [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) method.

**NOTE**: Video files have a `State` field in the File API. When a video is uploaded, it will be in the `PROCESSING` state until it is ready for inference. Only `ACTIVE` files can be used for model inference.

In [ ]:
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

### Prompt with a video and text

Once the uploaded video is in the `ACTIVE` state, you can make `GenerateContent` requests that specify the File API URI for that video. Select the generative model and provide it with the uploaded video and a text prompt.

In [ ]:
# Create the prompt.
prompt = "Summarize this video. Then create a quiz with answer key based on the information in the video."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})

# Print the response, rendering any Markdown
Markdown(response.text)

### Refer to timestamps in the content

You can use timestamps of the form `MM:SS` to refer to specific moments in the video.

In [ ]:
# Create the prompt.
prompt = "What are the examples given at 01:05 and 01:19 supposed to show us?"

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
print(response.text)

### Transcribe video and provide visual descriptions

If the video is not fast-paced (given that frames are sampled at 1 per second), it's possible to transcribe the video with visual descriptions for each shot.

In [ ]:
# Create the prompt.
prompt = "Transcribe the audio, giving timestamps. Also provide visual descriptions."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
print(response.text)

## List files

You can list all uploaded files and their URIs using `files.list_files()`.

In [ ]:
# List all files
for file in genai.list_files():
    print(f"{file.display_name}, URI: {file.uri}")

## Delete files

Files are automatically deleted after 2 days. You can also manually delete them using `files.delete()`.

In [ ]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')